In [ ]:
import pandas as pd
import re

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(5)

In [ ]:
months = [
    "Jan([ua]{2}ry)?", "Feb(ruary)?", "Mar(?:ch)?", "Apr(?:il)?",
    "May", "Jun(?:e)?", "July?", "Aug(ust)?",
    "Sep(?:tember)?", "Oct(?:ober)?", "Nov(?:eme?ber)?", "Dec(?:eme?ber)?"
]

regexs = [
    (r'(?P<month>[01]?\d)/(?P<day>[0123]?\d)/(?P<year>\d{2}|\d{4})', {}),
    (r'(?P<month>[01]?\d)-(?P<day>[0123]?\d)-(?P<year>\d{2}|\d{4})', {}),
    (r'(?P<month>[01]\d)(?P<day>[0123]\d)(?P<year>\d{2}|\d{4})', {}),
    (r'(?P<month>[01]?\d)/(?P<year>\d{2}|\d{4})', {"day": 1}),
] + [
    (f"(?P<day>[0123]\d) ({month}) (?P<year>\d\d(\d\d)?)", {"month": month_num})
    for month, month_num in enumerate(months, 1)
] + [
    (f"(({month})\W? (?P<day>[0123]\d)\W? (?P<year>\d\d(\d\d)?))", {"month": month_num})
    for month, month_num in enumerate(months, 1)
] + [
    (f"({month})\W? (?P<year>\d\d(\d\d)?)", {"month": month_num})
    for month, month_num in enumerate(months, 1)
] + [
    ("(?P<year>\d{4})", {"day": 1, "month": 1})
]
regexs = [(re.compile(regex, re.IGNORECASE), additional_info) for (regex, additional_info) in regexs]

In [ ]:
def parse_str(s: str):
    for regex, add_info in regexs:
        match = regex.search(s)
    
    #return [12, s, "lala"]
    return {"a": 12, "b": s, "c": "lala"}

for regex, add_info in regexs:
    match = regex.search(text_string)
    if match:
        break


        
#x = df.head().map(parse_str)
#pd.DataFrame.from_records(list(x))

In [ ]:
mdy = df.str.extractall(r'(?P<date>(?P<month>[01]?\d)[/-]?(?P<day>[0123]?\d)[/-](?P<year>\d{2}|\d{4}))')
mdy = mdy[mdy.date.str.match("|".join(regexs))]
mdy.head(5)

In [ ]:
months = "(" + "|".join([
    "Jan(?:[ua]{2}ry)?", #aury
    "Feb(?:ruary)?", "Mar(?:ch)?", "Apr(?:il)?",
    "May", "Jun(?:e)?", "Jul(?:y)?", "Aug(?:ust)?",
    "Sep(?:tember)?", "Oct(?:ober)?", "(Nov|Dec)(?:eme?ber)?"
]) + ")"

regexs = [
    r'((?:[01]?\d)/([0123]?\d)/(\d{2}|\d{4}))',
    r'((?:[01]?\d)/(\d{2}|\d{4}))',
    r'((?:[01]?\d)-([0123]?\d)-(\d{2}|\d{4}))',
    r'((?:[01]\d)([0123]\d)(\d{2}|\d{4}))',
    f"((?:[0123]\d) {months} (\d\d(\d\d)?))",
    f"({months}\W? ([0123]\d)\W? (\d\d(\d\d)?))",
    f"({months}\W? (\d\d(\d\d)?))",
    "(\d{4})"
]
test_string = "vent in Decemeber 1978"

regex_str = "|".join(regexs)
regex = f"(?P<date>{regex_str})"
print(regex_str)

print(re.compile(regex, re.IGNORECASE).search(test_string))

mdy = df.str.extractall(regex)[["date"]]
mdy.tail()

In [ ]:
missing_row = list(set(range(df.shape[0])) - {x for (x, y) in mdy.index})
print("missing :", missing_row)
for i in missing_row:
    print(df.iloc[i])

In [ ]:
duplicated_row = {x for (x, y) in mdy.index if y > 0}
print()
print()
print("duplicated :", duplicated_row)
for i in duplicated_row:
    print(i, df.iloc[i])


print()
print()
print("duplicated rows")
for idx, row in mdy.T.iteritems():
    if idx[0] in duplicated_row:
        print(row)